<a href="https://www.kaggle.com/code/hydramst/water-pollution-lr?scriptVersionId=97774899" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/water-pollution/Submission.csv
/kaggle/input/water-pollution/Target.csv
/kaggle/input/water-pollution/Train.csv
/kaggle/input/water-pollution/Test.csv


In [2]:
DF_train = pd.read_csv('/kaggle/input/water-pollution/Train.csv', delimiter = ',')
DF_test = pd.read_csv('/kaggle/input/water-pollution/Test.csv', delimiter = ',')
Target = pd.read_csv('/kaggle/input/water-pollution/Target.csv', delimiter = ',')
Submission = pd.read_csv('/kaggle/input/water-pollution/Submission.csv', delimiter = ',')


In [3]:
df = pd.concat([DF_train, DF_test])
df = df.drop(columns = ['Unnamed: 0','period', 'venue', 'rate'], axis =1)
df = df#.join(Target['polution'])

## view categorical and numerical columns

In [4]:
 

cat_columns = []
num_columns = []

for column_name in df.columns:
    if (df[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

categorical columns:	  ['code', 'id', 'Country'] 
 len =  3
numerical columns:	  ['year', 'tourists', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 
 len =  11


## find duplicated rows

In [5]:

 
#df = df.drop_duplicates() 
df = df.reset_index(drop=True)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2440 entries, 0 to 2439
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   code             2440 non-null   object 
 1   year             2440 non-null   int64  
 2   id               2440 non-null   object 
 3   Country          2440 non-null   object 
 4   tourists         2440 non-null   float64
 5   food             2440 non-null   float64
 6   glass            2440 non-null   float64
 7   metal            2440 non-null   float64
 8   other            2440 non-null   float64
 9   paper            2440 non-null   float64
 10  plastic          2440 non-null   float64
 11  leather          2440 non-null   float64
 12  green_waste      2440 non-null   float64
 13  waste_recycling  2440 non-null   float64
dtypes: float64(10), int64(1), object(3)
memory usage: 267.0+ KB


## OHE of categorical features

In [7]:
df = pd.get_dummies(df)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2440 entries, 0 to 2439
Columns: 1541 entries, year to Country_United Kingdom
dtypes: float64(10), int64(1), uint8(1530)
memory usage: 3.8 MB


In [9]:
# Split back to test and train 

len_test = len(DF_test)
train = df.iloc[:-len_test]
test = df.iloc[-len_test:]
print(DF_train.shape[0], train.shape[0], DF_test.shape[0], test.shape[0])

2318 2318 122 122


In [10]:
X, y = train.values, Target['polution'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Ridge LR

In [11]:

model = Ridge(alpha = 0.00005, max_iter=10000)

model.fit(X_train, y_train)

B0=model.intercept_
B=model.coef_

line='{:.2f}'.format(B0)
sign=['+','-']

features_names = df.columns
for p,(fn,b) in enumerate(zip(features_names,B)):
  if np.abs(b)>0:
    line=line+sign[int(0.5*(np.sign(b)-1))]+'{:.2f}*'.format(np.abs(b))+fn



y_predict=model.predict(X_test)

print('Ошибка на тестовых данных')
print('MSE: %.1f' % mse(y_test,y_predict))
print('RMSE: %.1f' % mse(y_test,y_predict,squared=False))
print('R2 : %.4f' %  r2_score(y_test,y_predict))

Ошибка на тестовых данных
MSE: 0.0
RMSE: 0.1
R2 : 0.4172


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.03275e-23): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


# Cross validation

In [12]:
scaler  = MinMaxScaler()
scaler.fit_transform(X_train)
X_train = scaler.transform(X_train) 
X_test  = scaler.transform(X_test)

In [13]:
scoring = {'R2': 'r2',
           '-MSE': 'neg_mean_squared_error',
           '-MAE': 'neg_mean_absolute_error',
           'Max': 'max_error'}


scores = cross_validate(model, X_train, y_train,
                      scoring=scoring, cv=ShuffleSplit(n_splits=5, random_state=42) )

print('Результаты Кросс-валидации')
DF_cv_linreg = pd.DataFrame(scores)
display(DF_cv_linreg)
print('\n')
print(DF_cv_linreg.mean()[2:])
print('\n')

Результаты Кросс-валидации


,fit_time,score_time,test_R2,test_-MSE,test_-MAE,test_Max
0,0.140791,0.001682,0.102971,-0.008477,-0.048241,-0.545864
1,0.190638,0.001594,0.401168,-0.008062,-0.048124,-0.483864
2,0.177824,0.008761,0.461281,-0.004529,-0.045163,-0.237212
3,0.178686,0.008490,0.287425,-0.007876,-0.044458,-0.792190
4,0.185665,0.002879,0.379922,-0.006622,-0.044935,-0.480110




test_R2      0.326553
test_-MSE   -0.007113
test_-MAE   -0.046184
test_Max    -0.507848
dtype: float64




In [14]:
model = Ridge(alpha=0.0001, max_iter=10000)

model.fit(X_train, y_train)

y_predict = model.predict(test)

Submission['polution'] = y_predict
Submission

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but Ridge was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


,idx,polution
0,402,-421389.394832
1,748,-421389.347695
2,763,-421389.347695
3,942,-134076.018198
4,1258,-421389.382504
...,...,...
117,19447,-153208.389157
118,19458,-421389.419581
119,19464,-421389.483296
120,19699,-421389.441970


In [15]:
! rm /kaggle/working/water-pollution/Submission.csv
! mkdir /kaggle/working/water-pollution/

rm: cannot remove '/kaggle/working/water-pollution/Submission.csv': No such file or directory


In [16]:
Submission.to_csv('/kaggle/working/water-pollution/Submission.csv',index=False)

In [17]:
! mkdir ~/.kaggle
! touch ~/.kaggle/kaggle.json
! echo '{"username":"","key": "" }' > ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [18]:
cat ~/.kaggle/kaggle.json

{"username":"","key": "" }


In [19]:
! kaggle competitions submit -f /kaggle/working/water-pollution/Submission.csv -m "Message" sf-matml-2022-regression

401 - Unauthorized
